# АНСАМБЛИ МОДЕЛЕЙ

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from jupyterthemes import jtplot
from sklearn.metrics import auc, roc_curve, roc_auc_score
%matplotlib inline
jtplot.style()
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.datasets import make_blobs
from sklearn.datasets import make_moons

# Домашняя работа

1. Теперь решаем задачу регрессии - предскажем цены на недвижимость. Использовать датасет https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data (train.csv)
2. Данных немного, поэтому необходимо использовать 10-fold кросс-валидацию для оценки качества моделей
3. Построить случайный лес, вывести важность признаков
4. Обучить стекинг как минимум 3х моделей, использовать хотя бы 1 линейную модель и 1 нелинейную
5. Для валидации модели 2-го уровня использовать отдельный hold-out датасет, как на занятии
6. Показать, что использование ансамблей моделей действительно улучшает качество (стекинг vs другие модели сравнивать на hold-out)
7. В качестве решения:
    Jupyter notebook с кодом, комментариями и графиками

In [2]:
data1 = pd.read_csv('train_home.csv')
data1.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


все пропущенные заполним

In [3]:
data1['Street'].value_counts()
data1.shape
#data1.info()
#data1['Alley'].isnull().sum()
data1.fillna(-999, inplace=True)
data1

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,-999,Reg,Lvl,AllPub,...,0,-999,-999,-999,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,-999,Reg,Lvl,AllPub,...,0,-999,-999,-999,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,-999,IR1,Lvl,AllPub,...,0,-999,-999,-999,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,-999,IR1,Lvl,AllPub,...,0,-999,-999,-999,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,-999,IR1,Lvl,AllPub,...,0,-999,-999,-999,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,-999,Reg,Lvl,AllPub,...,0,-999,-999,-999,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,-999,Reg,Lvl,AllPub,...,0,-999,MnPrv,-999,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,-999,Reg,Lvl,AllPub,...,0,-999,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,-999,Reg,Lvl,AllPub,...,0,-999,-999,-999,0,4,2010,WD,Normal,142125


In [4]:
data_y = data1['SalePrice']
data_X = data1.drop(['SalePrice'], axis=1)

#data.head()
data_y.shape, #data_X.info()


((1460,),)

выделим категориальные признаки для дальнейшей кодировки, заполним пустые

In [5]:
categorial_feat = list(data_X.dtypes[data_X.dtypes ==object].index)
#data_X[cat_feat] = data_X[cat_feat].fillna('NaN')
data_X[categorial_feat] = data_X[categorial_feat].fillna(-999)
usually_feat = [f for f in data_X if f not in (categorial_feat + ['Id'])]
# usually_feat
categorial_unique = data_X[categorial_feat].nunique()
usually_unique = data_X[usually_feat].nunique()
data_X[usually_feat] = data_X[usually_feat].fillna(-999)
#print(categorial_unique)
#data_X['Alley'].fillna('-999', inplace=True)
data_X[usually_feat]

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,548,0,61,0,0,0,0,0,2,2008
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,460,298,0,0,0,0,0,0,5,2007
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,608,0,42,0,0,0,0,0,9,2008
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,642,0,35,272,0,0,0,0,2,2006
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,836,192,84,0,0,0,0,0,12,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,62.0,7917,6,5,1999,2000,0.0,0,0,...,460,0,40,0,0,0,0,0,8,2007
1456,20,85.0,13175,6,6,1978,1988,119.0,790,163,...,500,349,0,0,0,0,0,0,2,2010
1457,70,66.0,9042,7,9,1941,2006,0.0,275,0,...,252,0,60,0,0,0,0,2500,5,2010
1458,20,68.0,9717,5,6,1950,1996,0.0,49,1029,...,240,366,0,112,0,0,0,0,4,2010


закодируем и разделим на X и y

In [32]:
#первый аргумент - вся талица, которую будем использовать, а второй - тольо то, что необходимо закодировать
X = pd.get_dummies(data_X, columns=categorial_feat)
d_train = data1.iloc[train_index]
d_test = data1.iloc[test_index]
dummy_test = pd.get_dummies(d_test[categorial_feat], columns=categorial_feat)
dummy_train = pd.get_dummies(d_train[categorial_feat], columns=categorial_feat)
X.shape

(1460, 305)

In [33]:
y = data_y
y.shape


(1460,)

построим лес деревьев регрессии, проверим, насколько хороша модель на тестовых

In [34]:
model = RandomForestRegressor(n_estimators=500, max_features=5)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model.fit(X_train, y_train)
y_pred_train, y_pred_test = model.predict(X_train), model.predict(X_test)
mean_squared_error(y_train, y_pred_train), mean_squared_error(y_test, y_pred_test)
r2_score(y_train, y_pred_train), r2_score(y_test, y_pred_test)                                                                   
                                                                        

(0.9733343896174143, 0.805599847936949)

проведем кросс-валидацию и проверим, насколько улучшились значения

In [35]:
skf = KFold(n_splits=10, random_state=None, shuffle=False)
train_metric, test_metric = [], []
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
    model = RandomForestRegressor(n_estimators=500, max_features=5)
    model.fit(X_train, y_train)
    y_pred_train, y_pred_test = model.predict(X_train), model.predict(X_test)
    mean_squared_error(y_train, y_pred_train), mean_squared_error(y_test, y_pred_test)
    train_metric.append(r2_score(y_train, y_pred_train))
    test_metric.append(r2_score(y_test, y_pred_test))
    print(r2_score(y_train, y_pred_train), r2_score(y_test, y_pred_test))
print(sum(train_metric)/len(train_metric))
print(sum(test_metric)/len(test_metric))

0.9749548452932566 0.888812682777468
0.9759638090404618 0.8317463931133473
0.9749108762743592 0.8753148735329656
0.975946897657043 0.7904284099852911
0.9749851183132499 0.7856487363647513
0.9743134317588077 0.821976952766252
0.9756823409264879 0.8633864745618297
0.9753521596728609 0.8486519211904018
0.9765429147874075 0.7054767146334673
0.9761678895189012 0.8471417410491215
0.9754820283242835
0.8258584899974896


оценим важность признаков

In [36]:
imp = pd.Series(model.feature_importances_)
imp.sort_values(ascending=False)

4      0.042678
16     0.041064
27     0.036019
13     0.034190
12     0.033682
         ...   
153    0.000000
227    0.000000
214    0.000000
138    0.000000
289    0.000000
Length: 305, dtype: float64

Обучить стекинг как минимум 3х моделей, использовать хотя бы 1 линейную модель и 1 нелинейную

In [44]:
new_model = RandomForestRegressor(n_estimators=10, max_depth=5, min_samples_leaf=20, max_features=0.5, n_jobs=-1)
X_train_lin, X_test_lin, y_train, y_test = train_test_split(X, y, test_size=0.2)
model.fit(X_train_lin, X_test_lin)
y_pred_train, y_pred_test = model.predict(X_train_lin), model.predict(X_test_lin)


ValueError: Number of labels=292 does not match number of samples=1168

In [38]:
y_pred_test_rf = new_model.predict(X_test_lin)

NameError: name 'X_test_lin' is not defined

In [39]:
from sklearn.preprocessing import StandardScaler

train_median = d_train[usually_feat].median()

X_train_lin = pd.concat([d_train[usually_feat].fillna(train_median),
                     d_train[usually_feat + categorial_feat].isnull().astype(np.int8).add_suffix('_NaN'),
                     dummy_train], axis=1)

X_test_lin = pd.concat([d_test[usually_feat].fillna(train_median),
                     d_test[usually_feat + categorial_feat].isnull().astype(np.int8).add_suffix('_NaN'),
                     dummy_test], axis=1)

scaler = StandardScaler()
scaler.fit(X_train_lin[usually_feat])

X_train_lin[usually_feat] = scaler.transform(X_train_lin[usually_feat])
X_test_lin[usually_feat] = scaler.transform(X_test_lin[usually_feat])

In [41]:
from sklearn.linear_model import LogisticRegression
clf_lr = LogisticRegression(penalty='l1', C=0.1)

clf_lr.fit(X_train_lin, y_train)

/Users/natalaurina/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/natalaurina/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [45]:
y_pred_lin_test = clf_lr.predict_proba(X_test_lin)[:, 1]
y_pred_lin_train = clf_lr.predict_proba(X_train_lin)[:, 1]

print('Train:')
show_auc(y_train, y_pred_lin_train, 'train')
print('Test:')
show_auc(y_test, y_pred_lin_test, 'test')
plt.legend();

ValueError: X has 305 features per sample; expecting 378

Будем строить линейную комбинацию логистической регрессии и случайного леса вида 

$y = \alpha y_1 + (1 - \alpha) y_2$

Параметр $\alpha$ переберем по сетке от 0 до 1, оценивая качество на тестовой выборке

In [47]:
aucs = []
alpha_space = np.linspace(0, 1, 100)
for alpha in alpha_space:
    y_pred_weight = alpha * y_pred_lin_test + (1 - alpha) * y_pred_test_rf
    aucs.append(show_auc(y_test, y_pred_weight, prin=False, plot_label=str(alpha)))
aucs = np.array(aucs)  
alpha * y_pred_lin_test + (1 - alpha) * y_pred_test_rf


NameError: name 'y_pred_lin_test' is not defined

In [ ]:
alpha * y_pred_test + (1 - alpha) * y_pred_test_rf

In [ ]:
max_ind = np.where(aucs == aucs.max())[0]
alpha = alpha_space[max_ind]

plt.plot(alpha_space, aucs)
plt.plot(alpha_space[max_ind], aucs[max_ind], 'o', c='r')
plt.xlabel('alpha')
plt.ylabel('auc')

#итоговое взвешенное предсказание
y_pred_weight = alpha * y_pred_test + (1 - alpha) * y_pred_test_rf

In [ ]:
print('Weighted:')
show_auc(y_test, y_pred_weight, 'weighted')
print('Log regression:')
show_auc(y_test, y_pred_test, 'LR')
print('Random forest:')
show_auc(y_test, y_pred_test_rf, 'RF')
plt.legend();
plt.xlim(0.2, 0.5)
plt.ylim(0.5, 0.8)

стэккинг

In [443]:
from sklearn.model_selection import StratifiedKFold

stack_cv = StratifiedKFold(n_splits=10, random_state=555)

meta_train = []
meta_test = []
col_names = []

print('LR features...')
meta_tr, meta_te = get_meta_features(clf_lr, X_train_lin, y_train, X_test_lin, stack_cv)

meta_train.append(meta_tr)
meta_test.append(meta_te)
col_names.append('lr_pred')

print('RF features...')
meta_tr, meta_te = get_meta_features(clf_rf, X_train, y_train, X_test, stack_cv)

meta_train.append(meta_tr)
meta_test.append(meta_te)
col_names.append('rf_pred')

LR features...


NameError: name 'X_train_lin' is not defined

In [ ]:
#стэккинг!!!!!
mlens.ensemble superlearner

In [340]:
scores

array([], dtype=float64)

In [339]:
#Чтобы в разы не увеличивать число признаков при построении dummy,
#будем использовать категориальные признаки с < 30 уникальных значений
cat_feat = list(cat_nunique[cat_nunique < 30].index)
# Создаем дамми-переменные для категорий
dummy_train = pd.get_dummies(d_train[cat_feat], columns=cat_feat)
dummy_test = pd.get_dummies(d_test[cat_feat], columns=cat_feat)

dummy_cols = list(set(dummy_train) & set(dummy_test))

dummy_train = dummy_train[dummy_cols]
dummy_test = dummy_test[dummy_cols]


# Заменяем пропуски на специальное значение -999, чтобы деревья могли их отличить
X_train = pd.concat([d_train[num_feat].fillna(-999),
                     dummy_train], axis=1)

X_test = pd.concat([d_test[num_feat].fillna(-999),
                     dummy_test], axis=1)
from sklearn.tree import DecisionTreeClassifier

clf_tree = DecisionTreeClassifier(max_depth=15, min_samples_leaf=20)
clf_tree.fit(X_train, y_train)
def show_auc(y, y_pred, plot_label='', prin=True):    
    fpr, tpr, _ = roc_curve(y, y_pred)
    auc_val = auc(fpr, tpr)
    if prin:
        print('ROC AUC: {0:.4f}'.format(auc_val))
    if plot_label:
        plt.plot(fpr, tpr, label=plot_label)
        plt.xlabel('FPR')
        plt.ylabel('TPR')
    return auc_val
y_pred_test = clf_tree.predict_proba(X_test)[:, 1]
y_pred_train = clf_tree.predict_proba(X_train)[:, 1]

print('Train:')
show_auc(y_train, y_pred_train, 'train')
         
print('Test:')
show_auc(y_test, y_pred_test, 'test')

KeyError: "None of [Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',\n       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',\n       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',\n       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',\n       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',\n       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',\n       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',\n       'MoSold', 'YrSold'],\n      dtype='object')] are in the [columns]"